# Build, test or integrate your planner

Welcome back, this is the second tutorial of InterSim. Please make sure that you have a proper environment setup for InterSim before moving forward. 

In this tutorial, you will learn the APIs to:
1. get the data your planner need in your planner
2. return a legit planning result to the simulator

You will also learn how to debug your planner's result per step with the help of your visualization tools. Let's get started.

## Create and Import Your Planner

To simulate in a closed-loop fashion, InterSim has a planenr for all environment agents `env_planner.py` and a planner for the ego vehicle `base_planenr.py`. 

The first step to create a new planner is to create a new python script in the folder `./plan`. The most easy way is to copy and start from the `dummpy_planner.py`. Follow the following steps to register and switch a new planner:

1. Rename your planner script at your preference, like `your_planenr.py`.
2. Rename your planner class, for example `class YourPlanner(EnvPlanner):`. It is compulsory to inherit from the EnvPlanner.
3. Import your planner at `drive.py`, for example `from plan.your_planner import YourPlanner`
4. Initialize your planner in the configure function at `drive.py` with a customized config value, like `your_planner_name`.
5. Change the parameter of `ego_planner` to your cusomized planner name, like `your_planner_name`.

Try to run the simulation again, now InterSim should you your new planner instead of the default ego_planner.

`python sim.py --max_scenarios 5`

## plan_ego function

You can see a minimal plan_ego function defined in the `dummy_planner.py`. In every step, as defined by frame_rate in the config, `drive.py` will call this function to aquire a new ego plan. Like the dummy_planner, you might need the following variables and functions for your planner.

### Basic Planning Variables

InterSim will initiate the planner with these variables:

- self.frame_rate: how many frames are there in the each second
- self.planning_from: planner start planning no later than this frame
- self.planning_to: planner will not plan after a specific frame
- self.planning horizon: planner plans a trajectory for x frames in the future
- self.total_frame: data_loader loads x frames and the simulation will terminate after x
- self.planning_interval: planner plans every x frames

You should change these variables from the config to make changes gloable.

### Basic Planning Infos

At every time step, the simulator will pass two parameters into plan_ego for planning:

1. current_state: this is a dictionary that holds all information of the current scenario
2. current_frame_idx: the current frame number for planning

You might need the following information for your planner to plan:

- ego_agent_id: query by `current_state['predicting']['ego_id'][1]`
- total_time_frame: query by `current_state['agent'][ego_agent_id]['pose'].shape[0]`
- my_current_pose: query by `current_state['agent'][ego_agent_id]['pose'][current_frame_idx - 1]`
- current_route: this is a list of lane_ids as the route for the ego to travel, query by `self.get_reroute_traj()`

### Interperlator

Planners has a default interperlator, `SudoInterpolator`. You can initialize the interperlator with an array of points (at least two points). Then you can get a trajectory from the path you initalize for a series of coming time steps by calling `self.get_trajectory_from_interpolator()`.

Note neither the interperlator nor the function that converts path into trajectories from the beta verion is perfect at all. Ideally, the interperpolator should consider a vehicle dynamic model and the get_trajectory function should apply a decent optimization method for better performance.

### Return the Ego Planning Result

Replace the future trajectory of the ego_agent by one line of codes like blow and the `planed_traj` is the trajectory you planned with a dimention of t*4. The pose is a t*4 dimensional array where t is the number of time steps. The 4 dimensinal state is $[x, y, z, yaw]$. Yaw is an angular value from $[-\pi, \pi]$.

```python
current_state['agent'][ego_agent_id]['pose'][current_frame_idx:planning_horizon +
                                             current_frame_idx, :] = planed_traj[:total_time_frame - 
                                                                                 current_frame_idx, :]
```

## Debug

Debugging planners can be exhausting considering the great complexity of the planning task itself, especially when you involve predictors. It is critical to debug your planners at every step with a clear visualization.

InterSim provides a convenient Python-based visualization tool to debug your planner at each step. Change the running_mode in config to 0 and the visualization window should pop-up like below. Our Python-based visualization tool works on Mac, Windows, and Ubuntu.

It is easy to debug your planner with the help of the Python-based visualization tool. Pass your planning trajectory, which should have a t*4 dimension, into the list for `trajectory_to_mark` in the `predicting` dictionary like below. And you should see a trajectory plot on the visualization for your plan at each step. You can imply this same trick when debuging your predictor by ploting your prediction results, as shown in the default `base_planner`.

InterSim will loop all trajectories in the `trajectory_to_mark`, plot them and clean them at each step after a planning interval.

<img src="./visual_snapshot.png"  width="500" height="500">